In [1]:
import asyncio
import csv
from datetime import datetime
import json
import time
import visualizer

import requests
import sys

# BUFF

In [7]:
def get_buff_json(url, hidden_prints:bool=False):
    headers = {
        "Accept-Language": "en-US",
        # "cookie": "session=1-KkV0mFdN6EI9S8ZGWA-Aldct4C_eK0uhmoUit1Y9KnJW2038331450", main
        "cookie": "session=1-iOHNuqGSjRWueamMaALM9ytyt7eoLLju47e2Wpb-RZ0z2029595640", # RenaGonca
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
        "Referer": "https://buff.163.com/market/dota2",
        "X-Requested-With": "XMLHttpRequest"
      }
    request = requests.get(url, headers=headers)
    while request.status_code != 200 and request.status_code == 429:
        time.sleep(10)
        if not hidden_prints: print("\033[91mError requesting again...")
        request = requests.get(url, headers=headers)
    print(request.text)
    return request

In [6]:
def bulk_buff_search(page:int, page_size:int=80, hidden_prints:bool=False):
    reference_time = datetime.now().strftime("%H:%M")

    url = f"https://buff.163.com/api/market/goods?game=csgo&page_num={page * page_size}&page_size=80&sort_by=sell_num.desc&sort_order=desc"
    
    request = get_buff_json(url)

    items = request.json()["data"]["items"]
    res = []
    for item in items:
        market_hash_name = item["market_hash_name"]
        good_id = item["id"]
        min_sell_order = round(float(item["sell_min_price"]) * 0.13, 2)
        sell_orders = item["sell_num"]
        buy_orders = item["buy_num"]
        max_buy_order = round(float(item["buy_max_price"]) * 0.13, 2)
        steam_reference_price = round(float(item["goods_info"]["steam_price_cny"])*0.13, 2)

        res.append([reference_time, market_hash_name, min_sell_order, sell_orders, max_buy_order, buy_orders, steam_reference_price])
        if not hidden_prints: print("\033[92m" + market_hash_name, min_sell_order, sell_orders, max_buy_order, buy_orders, steam_reference_price, "\033[0m")

    return res
bulk_buff_search(10)


'https://buff.163.com/api/market/goods?game=csgo&page_num=800&page_size=80&sort_by=sell_num.desc&sort_order=desc'

# STEAM